In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,recall_score,precision_score
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
import copy
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import LancasterStemmer

In [2]:
df = pd.read_csv('C:/Users/Paladian/Downloads/training.1600000.processed.noemoticon.csv',delimiter=',', encoding='ISO-8859-1')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048572 entries, 0 to 1048571
Data columns (total 6 columns):
 #   Column              Non-Null Count    Dtype 
---  ------              --------------    ----- 
 0   polarity of tweet   1048572 non-null  int64 
 1   id of the tweet     1048572 non-null  int64 
 2   date of the tweet   1048572 non-null  object
 3   query               1048572 non-null  object
 4   user                1048572 non-null  object
 5   text of the tweet   1048572 non-null  object
dtypes: int64(2), object(4)
memory usage: 48.0+ MB


In [4]:
df.head()

,polarity of tweet,id of the tweet,date of the tweet,query,user,text of the tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [5]:
df["text"]=df["text of the tweet\xa0"]
df["Label"]=df["polarity of tweet\xa0"]
df["Originaltext"]=df["text of the tweet\xa0"]
datadf=df[["Label","text"]]
datadf.columns.values

array(['Label', 'text'], dtype=object)

In [6]:
stuff_to_be_removed = list(stopwords.words('english'))+list(punctuation)
stemmer = LancasterStemmer()
corpus = datadf['text'].tolist()
print(len(corpus))
print(corpus[0])

1048572
is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!


In [7]:
final_corpus = []
final_corpus_joined = []
for i in datadf.index:
    text = re.sub('[^a-zA-Z]', ' ', df['text'][i])
    text = text.lower()
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    text=re.sub("(\\d|\\W)+"," ",text)
    text = text.split()
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text  if not word in stuff_to_be_removed] 
    text1 = " ".join(text)
    final_corpus.append(text)
    final_corpus_joined.append(text1)

In [8]:
newdata = pd.DataFrame()
newdata["text"] = final_corpus_joined
newdata["Label"] = datadf["Label"].values

In [15]:
newdata["Label"]= newdata["Label"].replace(to_replace = 4, value = 1)

In [17]:
newdata.tail()

,text,Label
1048567,grandma making dinenr mum,1
1048568,mid morning snack time bowl cheese noodle yum,1
1048569,shadela say like terminiator movie come like word,1
1048570,destinyhope im great thaanks wbuu,1
1048571,cant wait til date weekend,1


In [18]:
X_train, X_test, y_train, y_test = train_test_split(newdata["text"], newdata["Label"], test_size=0.33, random_state=0)

In [19]:
print(X_train)

351360    one else sonicate vial stopper slip drop sampl...
716114                   jessemccartney awe thats far still
470702    trying continue chimera minute project prevent...
508989    another car got towed going spree look like ne...
175763                          janelliebeans even remember
                                ...                        
963395    awesome guy life hr away getting dick sucked s...
117952    elle macpherson bra brilliant need money buy g...
435829                amber x ive got wait another month xx
305711    thisgoeshere june th excited co ill le time us...
985772                                 georgieboo aww thank
Name: text, Length: 702543, dtype: object


In [20]:
vect = TfidfVectorizer().fit(X_train)

In [21]:
X_train_vectorized = vect.transform(X_train)

In [27]:
model = LogisticRegression(max_iter=10000)
model.fit(X_train_vectorized,y_train)

LogisticRegression(max_iter=10000)

In [28]:
predictions = model.predict(vect.transform(X_test))

In [32]:
test_review = vect.transform(["I absolutely love this product. This product is good and everyone should buy it"])
model.predict(test_review)

array([0], dtype=int64)

In [ ]:
test_review = vect.transform(["I bought it for a friend late. Delivered late. Packet not in good condit"])
model.predict(test_review)

In [30]:
nb_report = accuracy_score(y_test,predictions)
print('Accuracy:',nb_report*100)

Accuracy: 83.63085174941985


In [31]:
metrics.confusion_matrix(y_test,predictions)

array([[248858,  15175],
       [ 41467,  40529]], dtype=int64)